In [6]:
import mlflow 
import os

current_path = os.getcwd()
mlflow.set_tracking_uri(f"file://{current_path}/mlruns")
mlflow.set_experiment("MLflow Quickstart")

import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# Load the Iris dataset
X, y = datasets.load_iris(return_X_y=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Define the model hyperparameters
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 8888,
}

import mlflow 

mlflow.sklearn.autolog()

with mlflow.start_run(run_name = "Iris_LogReg_Denemesi"):
    lr = LogisticRegression(**params)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    mlflow.log_metric("test_accuracy", acc)

    print(f"Model eğitildi! Skor: {acc}")
    

/opt/anaconda3/envs/newUmut/lib/python3.10/site-packages/mlflow/tracking/_tracking_service/utils.py:177: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance. For migrating existing data, https://github.com/mlflow/mlflow-export-import can be used.
  return FileStore(store_uri, store_uri)
Traceback (most recent call last):
  File "/opt/anaconda3/envs/newUmut/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 379, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "/opt/anaconda3/envs/newUmut/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 477, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/opt/ana

Model eğitildi! Skor: 1.0


In [17]:
mlflow.set_experiment("Hyperparameter Tuning Experiment")

import mlflow
import mlflow.sklearn
import optuna
import sklearn.ensemble
import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing

X, y = fetch_california_housing(return_X_y=True)
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state = 0)

def objective(trial):
    with mlflow.start_run(nested=True, run_name = f"trial_{trial.number}") as cr:
        rf_max_depth = trial.suggest_int("rf_max_depth", 2, 32)
        rf_n_estimators = trial.suggest_int("rf_n_estimators", 50, 300, step = 10)
        rf_max_features = trial.suggest_float("rf_max_features", 0.2, 1.0)

        params = {
            "max_depth": rf_max_depth,
            "n_estimators": rf_n_estimators,
            "max_features": rf_max_features
        }

        mlflow.log_params(params)

        regressor_obj = sklearn.ensemble.RandomForestRegressor(**params)
        regressor_obj.fit(X_train, y_train)

        y_pred = regressor_obj.predict(X_val)
        error = sklearn.metrics.mean_squared_error(y_val, y_pred)

        mlflow.log_metrics({"mse": error})

        mlflow.sklearn.log_model(regressor_obj, name="model")

        trial.set_user_attr("run_id", cr.info.run_id)

        return error 

Traceback (most recent call last):
  File "/opt/anaconda3/envs/newUmut/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 379, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "/opt/anaconda3/envs/newUmut/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 477, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/opt/anaconda3/envs/newUmut/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 1662, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "/opt/anaconda3/envs/newUmut/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 1655, in _read_helper
    result = read_yaml(root, file_name)
  File "/opt/anaconda3/envs/newUmut/lib/python3.10/site-packages/mlflow/utils/yaml_utils.py", line 104, in read_yaml
    raise MissingConfigException(f"Yaml file '{file_path}' does not exist.")
mlflow.e

In [19]:
with mlflow.start_run(run_name="study") as run:
    n_trials = 10
    mlflow.log_param("n_trials", n_trials)

    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials = n_trials)

    mlflow.log_params(study.best_trial.params)
    mlflow.log_metrics({"best_error": study.best_value})
    
    best_run_id = study.best_trial.user_attrs.get("run_id")
    if best_run_id:
        mlflow.log_param("best_trial_run_id", best_run_id)

print("Optimizasyon bitti, MLflow UI'ı kontrol edebilirsin.")

[I 2025-12-21 17:47:28,970] A new study created in memory with name: no-name-f80e5673-396c-42a9-8a31-4529feea3ece
[I 2025-12-21 17:47:32,004] Trial 0 finished with value: 0.8462426758367869 and parameters: {'rf_max_depth': 2, 'rf_n_estimators': 210, 'rf_max_features': 0.34948750094065595}. Best is trial 0 with value: 0.8462426758367869.
[I 2025-12-21 17:47:44,445] Trial 1 finished with value: 0.26661088919608367 and parameters: {'rf_max_depth': 18, 'rf_n_estimators': 200, 'rf_max_features': 0.9142906122312789}. Best is trial 1 with value: 0.26661088919608367.
[I 2025-12-21 17:47:53,126] Trial 2 finished with value: 0.26279632777989204 and parameters: {'rf_max_depth': 29, 'rf_n_estimators': 130, 'rf_max_features': 0.7897580349536273}. Best is trial 2 with value: 0.26279632777989204.
[I 2025-12-21 17:48:02,633] Trial 3 finished with value: 0.2774378118356763 and parameters: {'rf_max_depth': 13, 'rf_n_estimators': 170, 'rf_max_features': 0.9761205423537063}. Best is trial 2 with value: 0.

Optimizasyon bitti, MLflow UI'ı kontrol edebilirsin.
